In [1]:
import os
import json
import shutil
from sklearn.model_selection import train_test_split
from pycocotools.coco import COCO
from PIL import Image
from glob import glob
import pandas as pd
from tqdm import tqdm
from ultralytics import YOLO

In [9]:
model = YOLO("yolov10x.pt")

In [10]:
model.train(
    data=r"./ai06-level1-project/yolo_dataset/data.yaml",
    epochs=500,
    imgsz=640,
    batch=4,
    lr0=1e-5,
    lrf=0.2,
    device=0,
    workers=2,
    amp=True,
    name="pill_yolo10x",
    pretrained=True,
    seed=42,
    optimizer='Adam',
    project='./ai06-level1-project/train_checkpoints'
)

Ultralytics 8.3.235 🚀 Python-3.11.14 torch-2.9.1+cu130 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./ai06-level1-project/yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=500, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=1e-05, lrf=0.2, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov10x.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=pill_yolo10x2, nbs=64, nms=False, opset=None, optimize=False, optimizer=Adam, overlap_mask=True, pa

KeyboardInterrupt: 

In [ ]:
model = YOLO("runs/detect/pill_yolo10x/weights/best.pt")

model.predict(
    source="./ai06-level1-project/test_cleaned",
    save=True,
    save_txt=True,
    save_conf=True,
    half=True,
    amp=True
)

In [8]:
#YOLO용으로 데이터 전처리

BASE = r"./ai06-level1-project/"
IMG_DIR = os.path.join(BASE, "train_output")
ANN_FILE = os.path.join(BASE, "train.json")
TEST_IMG_DIR = os.path.join(BASE, "test_images")

OUT_DIR = os.path.join(BASE, "yolo_dataset")

os.makedirs(os.path.join(OUT_DIR, "images/train"), exist_ok=True)
os.makedirs(os.path.join(OUT_DIR, "images/val"), exist_ok=True)
os.makedirs(os.path.join(OUT_DIR, "labels/train"), exist_ok=True)
os.makedirs(os.path.join(OUT_DIR, "labels/val"), exist_ok=True)

with open(ANN_FILE, "r", encoding="utf-8") as f:
    dataset = json.load(f)

coco = COCO()
coco.dataset = dataset
coco.createIndex()

names = [coco.cats[k]["name"] for k in sorted(coco.cats.keys())]
class_map={v['name']:k for k,v in coco.cats.items()}

creating index...
index created!


In [10]:
PRED_LABEL_DIR = "runs/detect/predict/labels"
IMAGE_DIR = "runs/detect/predict"
OUTPUT_CSV = os.path.join("./ai06-level1-project","submission.csv")

rows = []
annotation_id = 1

txt_files = glob(os.path.join(PRED_LABEL_DIR, "*.txt"))

for txt_path in txt_files:
    fname = os.path.basename(txt_path).replace(".txt", ".jpg")
    img_path = os.path.join(IMAGE_DIR, fname)

    if not os.path.exists(img_path):
        continue

    img = Image.open(img_path)
    W, H = img.size

    image_id = int(os.path.splitext(fname)[0])

    with open(txt_path, "r") as f:
        lines = f.readlines()

    for line in lines:
        cls, x, y, w, h, conf = map(float, line.split())

        bbox_x = (x - w/2) * W
        bbox_y = (y - h/2) * H
        bbox_w = w * W
        bbox_h = h * H

        rows.append({
            "annotation_id": annotation_id,
            "image_id": image_id,
            "category_id": class_map[names[int(cls)]],
            "bbox_x": int(bbox_x),
            "bbox_y": int(bbox_y),
            "bbox_w": int(bbox_w),
            "bbox_h": int(bbox_h),
            "score": round(conf, 4)
        })

        annotation_id += 1

df = pd.DataFrame(rows)
df.to_csv(OUTPUT_CSV, index=False)
print("CSV saved:", OUTPUT_CSV)

CSV saved: ./ai06-level1-project/submission.csv
